# Chapter 8: Robustness and Performance in Python

This notebook covers essential patterns and practices for writing robust, maintainable Python code with optimal performance characteristics.

## Item 65: Take Advantage of Each Block in try/except/else/finally

Python's exception handling constructs provide four distinct blocks, each serving specific purposes in error management and resource cleanup.

### The finally Block

The `finally` block executes cleanup code regardless of whether exceptions occur. This ensures resources are properly released even when errors arise.

In [ ]:
def try_finally_example(filename):
    print('* Opening file')
    handle = open(filename, encoding='utf-8')  # Maybe OSError
    try:
        print('* Reading data')
        return handle.read()  # Maybe UnicodeDecodeError
    finally:
        print('* Calling close()')
        handle.close()  # Always runs after try block

The `finally` block executes even when exceptions propagate upward. This demonstrates proper resource cleanup:

In [ ]:
# Create a test file with invalid UTF-8 data
filename = 'random_data.txt'
with open(filename, 'wb') as f:
    f.write(b'\xf1\xf2\xf3\xf4\xf5')  # Invalid utf-8

try:
    data = try_finally_example(filename)
except UnicodeDecodeError as e:
    print(f"\nCaught exception: {e.__class__.__name__}")
    print("Notice that close() was called before the exception propagated")

When the file doesn't exist, the `finally` block is skipped entirely because `open()` occurs before the try block:

In [ ]:
try:
    try_finally_example('does_not_exist.txt')
except FileNotFoundError as e:
    print(f"Caught: {e.__class__.__name__}")
    print("The finally block never executed")

### The else Block

The `else` block executes when the try block succeeds without exceptions. This clarifies exception handling boundaries and isolates potential error sources.

In [ ]:
import json

def load_json_key(data, key):
    try:
        print('* Loading JSON data')
        result_dict = json.loads(data)  # May raise ValueError
    except ValueError as e:
        print('* Handling ValueError')
        raise KeyError(key) from e
    else:
        print('* Looking up key')
        return result_dict[key]  # May raise KeyError

In the successful case, both the try and else blocks execute:

In [ ]:
result = load_json_key('{"foo": "bar"}', 'foo')
print(f"\nResult: {result}")
assert result == 'bar'

When JSON decoding fails, the except block handles the error:

In [ ]:
try:
    load_json_key('{"foo": bad payload', 'foo')
except KeyError as e:
    print(f"\nCaught KeyError: {e}")
    print("The ValueError was transformed into a KeyError")

Key lookup failures in the else block propagate naturally without special handling:

In [ ]:
try:
    load_json_key('{"foo": "bar"}', 'does not exist')
except KeyError as e:
    print(f"Caught KeyError from else block: {e}")

### Everything Together: try/except/else/finally

Combining all four blocks provides comprehensive error handling for complex operations requiring setup, processing, success actions, and cleanup.

In [ ]:
UNDEFINED = object()

def divide_json(path):
    print('* Opening file')
    handle = open(path, 'r+')  # May raise OSError
    try:
        print('* Reading data')
        data = handle.read()  # May raise UnicodeDecodeError
        print('* Loading JSON data')
        op = json.loads(data)  # May raise ValueError
        print('* Performing calculation')
        value = (
            op['numerator'] /
            op['denominator'])  # May raise ZeroDivisionError
    except ZeroDivisionError as e:
        print('* Handling ZeroDivisionError')
        return UNDEFINED
    else:
        print('* Writing calculation')
        op['result'] = value
        result = json.dumps(op)
        handle.seek(0)  # May raise OSError
        handle.write(result)  # May raise OSError
        return value
    finally:
        print('* Calling close()')
        handle.close()  # Always runs

Successful execution path (try → else → finally):

In [ ]:
temp_path = 'random_data.json'

with open(temp_path, 'w') as f:
    f.write('{"numerator": 1, "denominator": 10}')

result = divide_json(temp_path)
print(f"\nResult: {result}")
assert result == 0.1

Handled exception path (try → except → finally, skipping else):

In [ ]:
with open(temp_path, 'w') as f:
    f.write('{"numerator": 1, "denominator": 0}')

result = divide_json(temp_path)
print(f"\nResult is UNDEFINED: {result is UNDEFINED}")
assert result is UNDEFINED

Unhandled exception path (try → finally → exception propagates):

In [ ]:
with open(temp_path, 'w') as f:
    f.write('{"numerator": 1 bad data')

try:
    divide_json(temp_path)
except json.JSONDecodeError as e:
    print(f"\nCaught {e.__class__.__name__}")
    print("Notice that close() was called before exception propagated")

Exception in else block still triggers finally:

In [ ]:
# This would simulate disk full error during write in else block
# The finally block would still execute to close the file handle
print("The finally block ensures cleanup even when else block fails")

### Key Principles

**try block**: Contains code that might raise exceptions you want to handle

**except block**: Handles specific exceptions from the try block

**else block**: Executes only on try block success, before finally. Isolates success-case logic from exception-prone code

**finally block**: Always executes for cleanup, regardless of exceptions. Runs after try, except, and else blocks complete

## Item 66: Consider contextlib and with Statements for Reusable try/finally Behavior

Context managers abstract try/finally patterns into reusable constructs. The `with` statement provides syntactic sugar for context management.

### Basic with Statement Usage

The with statement replaces explicit try/finally constructions:

In [ ]:
from threading import Lock

lock = Lock()

# Modern approach with 'with'
with lock:
    print("This code runs while holding the lock")

print("\nEquivalent to:")

# Traditional approach
lock.acquire()
try:
    print("This code runs while holding the lock")
finally:
    lock.release()

### Creating Context Managers with contextlib

The `contextmanager` decorator transforms generator functions into context managers without requiring `__enter__` and `__exit__` methods.

In [ ]:
import logging
from contextlib import contextmanager

def my_function():
    logging.debug('Some debug data')
    logging.error('Error log here')
    logging.debug('More debug data')

@contextmanager
def debug_logging(level):
    logger = logging.getLogger()
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield  # Point where 'with' block executes
    finally:
        logger.setLevel(old_level)

The context manager temporarily modifies logging levels:

In [ ]:
# Default logging level is WARNING
print("Outside debug context:")
my_function()

print("\nInside debug context:")
with debug_logging(logging.DEBUG):
    my_function()

print("\nAfter debug context:")
my_function()

### Using with Targets

Context managers can yield values that become available via the `as` clause:

In [ ]:
# File handling with context manager
with open('my_output.txt', 'w') as handle:
    handle.write('This is some data!')

print("File was automatically closed after with block")

Creating custom context managers that yield values:

In [ ]:
@contextmanager
def log_level(level, name):
    logger = logging.getLogger(name)
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield logger  # Provides logger as target
    finally:
        logger.setLevel(old_level)

with log_level(logging.DEBUG, 'my-log') as logger:
    logger.debug(f'This is a message for {logger.name}!')
    logging.debug('This will not print')  # Default logger still at WARNING

After exiting the context, the logger returns to its default level:

In [ ]:
logger = logging.getLogger('my-log')
logger.debug('Debug will not print')
logger.error('Error will print')

Context managers provide state isolation by decoupling context creation from context usage:

In [ ]:
with log_level(logging.DEBUG, 'other-log') as logger:
    logger.debug(f'This is a message for {logger.name}!')
    logging.debug('This will not print')

## Item 67: Use datetime Instead of time for Local Clocks

Python provides two approaches for time zone conversions. The `time` module is platform-dependent and error-prone. The `datetime` module with `pytz` provides reliable cross-platform time zone handling.

### The time Module (Avoid)

The `time` module converts between UTC timestamps and local time:

In [ ]:
import time

now = 1552774475
local_tuple = time.localtime(now)
time_format = '%Y-%m-%d %H:%M:%S'
time_str = time.strftime(time_format, local_tuple)
print(f"Local time: {time_str}")

Converting back from local time to UTC:

In [ ]:
time_tuple = time.strptime(time_str, time_format)
utc_now = time.mktime(time_tuple)
print(f"UTC timestamp: {utc_now}")

The `time` module has severe platform-dependent limitations. Time zone support varies by operating system, making it unreliable for cross-platform applications.

### The datetime Module (Recommended)

The `datetime` module provides consistent time zone operations across platforms:

In [ ]:
from datetime import datetime, timezone

now = datetime(2019, 3, 16, 22, 14, 35)
now_utc = now.replace(tzinfo=timezone.utc)
now_local = now_utc.astimezone()
print(f"Local time: {now_local}")

Converting local time back to UTC timestamp:

In [ ]:
time_str = '2019-03-16 15:14:35'
now = datetime.strptime(time_str, time_format)
time_tuple = now.timetuple()
utc_now = time.mktime(time_tuple)
print(f"UTC timestamp: {utc_now}")

### Working with Multiple Time Zones Using pytz

The `pytz` library provides comprehensive time zone support. Always convert to UTC first, perform operations, then convert to local time:

In [ ]:
try:
    import pytz
    
    # Convert NYC time to UTC
    arrival_nyc = '2019-03-16 23:33:24'
    nyc_dt_naive = datetime.strptime(arrival_nyc, time_format)
    eastern = pytz.timezone('US/Eastern')
    nyc_dt = eastern.localize(nyc_dt_naive)
    utc_dt = pytz.utc.normalize(nyc_dt.astimezone(pytz.utc))
    print(f"UTC time: {utc_dt}")
    
    # Convert UTC to San Francisco time
    pacific = pytz.timezone('US/Pacific')
    sf_dt = pacific.normalize(utc_dt.astimezone(pacific))
    print(f"San Francisco time: {sf_dt}")
    
    # Convert UTC to Nepal time
    nepal = pytz.timezone('Asia/Katmandu')
    nepal_dt = nepal.normalize(utc_dt.astimezone(nepal))
    print(f"Nepal time: {nepal_dt}")
    
except ImportError:
    print("pytz not installed. Install with: pip install pytz")

### Best Practices for Time Zone Handling

**Always use UTC internally**: Store and process all times in UTC

**Convert to local time only for display**: Perform the conversion as the final step before presentation

**Use datetime with pytz**: Avoid the `time` module for anything beyond simple UTC to local conversions

**Follow the conversion pattern**: Local → UTC → Operations → UTC → Local

## Item 68: Make pickle Reliable with copyreg

The `pickle` module serializes Python objects but has limitations when class definitions change. The `copyreg` module provides mechanisms for backward-compatible serialization.

### Basic pickle Usage

Pickle serializes object state for later reconstruction:

In [ ]:
import pickle

class GameState:
    def __init__(self):
        self.level = 0
        self.lives = 4

state = GameState()
state.level += 1
state.lives -= 1
print(f"Original state: {state.__dict__}")

# Serialize to file
state_path = 'game_state.bin'
with open(state_path, 'wb') as f:
    pickle.dump(state, f)

# Deserialize from file
with open(state_path, 'rb') as f:
    state_after = pickle.load(f)
print(f"Restored state: {state_after.__dict__}")

### The Problem: Adding Fields

When you add fields to a class, old pickled objects lack those attributes:

In [ ]:
class GameState:
    def __init__(self):
        self.level = 0
        self.lives = 4
        self.points = 0  # New field

# Serialize with new definition
state = GameState()
serialized = pickle.dumps(state)
state_after = pickle.loads(serialized)
print(f"New serialization: {state_after.__dict__}")

# But old pickled data is missing the new field
with open(state_path, 'rb') as f:
    state_after = pickle.load(f)
print(f"Old serialization: {state_after.__dict__}")
print(f"Missing 'points' field: {'points' not in state_after.__dict__}")

### Solution: Default Attribute Values with copyreg

Use constructor default arguments to ensure all attributes exist after unpickling:

In [ ]:
import copyreg

class GameState:
    def __init__(self, level=0, lives=4, points=0):
        self.level = level
        self.lives = lives
        self.points = points

def pickle_game_state(game_state):
    kwargs = game_state.__dict__
    return unpickle_game_state, (kwargs,)

def unpickle_game_state(kwargs):
    return GameState(**kwargs)

copyreg.pickle(GameState, pickle_game_state)

Now serialization preserves all attributes:

In [ ]:
state = GameState()
state.points += 1000
serialized = pickle.dumps(state)
state_after = pickle.loads(serialized)
print(f"Serialized state: {state_after.__dict__}")

Adding new fields works seamlessly with default values:

In [ ]:
class GameState:
    def __init__(self, level=0, lives=4, points=0, magic=5):
        self.level = level
        self.lives = lives
        self.points = points
        self.magic = magic  # New field

# Old serialized data gets default value for new field
state_after = pickle.loads(serialized)
print(f"Old data with new field: {state_after.__dict__}")

### Versioning Classes

For backward-incompatible changes like removing fields, use version numbers:

In [ ]:
class GameState:
    def __init__(self, level=0, points=0, magic=5):
        self.level = level
        self.points = points
        self.magic = magic
        # Removed 'lives' field

def pickle_game_state(game_state):
    kwargs = game_state.__dict__
    kwargs['version'] = 2  # Mark as version 2
    return unpickle_game_state, (kwargs,)

def unpickle_game_state(kwargs):
    version = kwargs.pop('version', 1)
    if version == 1:
        del kwargs['lives']  # Remove obsolete field
    return GameState(**kwargs)

copyreg.pickle(GameState, pickle_game_state)

Old data deserializes successfully by removing obsolete fields:

In [ ]:
# Deserialize old version 1 data
state_after = pickle.loads(serialized)
print(f"Version 1 data migrated: {state_after.__dict__}")
print(f"'lives' field removed: {'lives' not in state_after.__dict__}")

### Stable Import Paths

Renaming classes breaks pickle unless you use copyreg to maintain stable unpickling paths:

In [ ]:
class BetterGameState:
    def __init__(self, level=0, points=0, magic=5):
        self.level = level
        self.points = points
        self.magic = magic

# Register with same unpickle function
copyreg.pickle(BetterGameState, pickle_game_state)

state = BetterGameState()
serialized = pickle.dumps(state)

# Serialized data references unpickle function, not class name
print(f"Serialized data references unpickle_game_state function")
print(f"Class can be renamed without breaking deserialization")

### pickle Security Warning

**Never unpickle data from untrusted sources.** Pickle can execute arbitrary code during deserialization. Use JSON for untrusted communication:

In [ ]:
import json

# Safe for untrusted data
data = {'level': 1, 'points': 1000, 'magic': 5}
serialized = json.dumps(data)
restored = json.loads(serialized)
print(f"JSON is safe for untrusted data: {restored}")

# Only use pickle between programs you control
print("\nUse pickle only for trusted internal communication")

## Item 69: Use decimal When Precision Is Paramount

IEEE 754 floating point numbers introduce rounding errors in financial and precision-critical calculations. The `Decimal` class provides fixed-point arithmetic with configurable precision.

### The Problem with float

Floating point arithmetic produces subtle errors:

In [ ]:
# Calculate phone call cost
rate = 1.45  # $/minute
seconds = 3*60 + 42  # 3 minutes 42 seconds
cost = rate * seconds / 60

print(f"Calculated cost: {cost}")
print(f"Expected cost: 5.365")
print(f"Error: {5.365 - cost}")

# Rounding makes it worse
rounded = round(cost, 2)
print(f"\nRounded cost: {rounded}")
print(f"Expected: 5.37 (rounded up)")
print(f"Actual: {rounded} (rounded down due to float error)")

### Solution: Decimal for Exact Arithmetic

The `Decimal` class provides exact decimal arithmetic:

In [ ]:
from decimal import Decimal

rate = Decimal('1.45')
seconds = Decimal(3*60 + 42)
cost = rate * seconds / Decimal(60)

print(f"Exact cost: {cost}")
print(f"No rounding error!")

### Decimal Construction: str vs float

Always use string constructor to avoid float precision loss:

In [ ]:
print("String constructor (exact):")
print(Decimal('1.45'))

print("\nFloat constructor (imprecise):")
print(Decimal(1.45))

print("\nInteger constructor (exact):")
print(Decimal(456))

### Controlled Rounding with quantize

The `quantize` method provides precise control over rounding behavior:

In [ ]:
from decimal import ROUND_UP

# Round up to nearest cent
cost = Decimal('5.365')
rounded = cost.quantize(Decimal('0.01'), rounding=ROUND_UP)
print(f"Rounded {cost} to {rounded}")

# Handle small costs correctly
rate = Decimal('0.05')
seconds = Decimal('5')
small_cost = rate * seconds / Decimal(60)
print(f"\nSmall cost: {small_cost}")

# round() would give 0.00
print(f"Using round(): {round(float(small_cost), 2)}")

# quantize() preserves minimum charge
rounded = small_cost.quantize(Decimal('0.01'), rounding=ROUND_UP)
print(f"Using quantize(): {rounded}")

### When to Use Decimal

**Financial calculations**: Monetary amounts require exact decimal arithmetic

**Precision-critical domains**: Scientific measurements, accounting, legal requirements

**Rounding control**: When you need specific rounding behaviors

**Display precision**: When output must match specific decimal places

For rational numbers with unlimited precision, consider the `Fraction` class from the `fractions` module.

## Item 70: Profile Before Optimizing

Python's dynamic nature makes performance intuition unreliable. Always measure before optimizing to identify actual bottlenecks rather than perceived ones.

### The Profiling Mindset

**Intuition is unreliable**: Dynamic features make performance counterintuitive

**Measure first**: Profile to identify real bottlenecks

**Focus optimization**: Target the biggest performance drains

**Follow Amdahl's Law**: Optimizing fast code wastes effort

### Example: Insertion Sort

Consider an intentionally inefficient insertion sort implementation:

In [ ]:
def insertion_sort(data):
    result = []
    for value in data:
        insert_value(result, value)
    return result

def insert_value(array, value):
    for i, existing in enumerate(array):
        if existing > value:
            array.insert(i, value)
            return
    array.append(value)

Test the implementation:

In [ ]:
from random import randint

data = [randint(0, 100) for _ in range(100)]
sorted_data = insertion_sort(data)
print(f"Sorted {len(data)} elements")
print(f"First 10: {sorted_data[:10]}")

### Using Python's Built-in Profiler

The `cProfile` module identifies performance bottlenecks. For notebook usage, we can use the `%%prun` magic command or manually profile functions.

In a production environment, you would run:

```python
import cProfile
profiler = cProfile.Profile()
profiler.runcall(insertion_sort, data)
```

The profiler output shows:
- Function call counts
- Total time spent in each function
- Cumulative time including subfunctions
- Time per call

This reveals where optimization efforts should focus.

### Key Profiling Principles

**Profile before optimizing**: Measure to find real bottlenecks, not assumed ones

**Use appropriate tools**: `cProfile` for function-level profiling, `line_profiler` for line-by-line analysis

**Focus on hotspots**: Optimize the slowest 20% of code that causes 80% of runtime

**Measure again**: Verify optimizations actually improve performance

**Maintain readability**: Don't sacrifice code clarity for marginal gains

## Summary: Robustness and Performance

This chapter covered essential patterns for building reliable, maintainable Python applications:

**Exception handling**: Use try/except/else/finally blocks appropriately for different error scenarios

**Context managers**: Abstract try/finally patterns with `contextlib` for cleaner, reusable code

**Time zones**: Always use `datetime` with `pytz` for reliable time zone conversions

**Serialization**: Make pickle reliable with `copyreg` for backward-compatible object persistence

**Precision arithmetic**: Use `Decimal` for financial calculations requiring exact decimal arithmetic

**Performance optimization**: Profile before optimizing to focus efforts on actual bottlenecks

These practices form the foundation for robust Python applications that handle errors gracefully, manage resources properly, and perform efficiently.